In [1]:
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd
import tensorflow as tf
import numpy as np
from keras import layers
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error,mean_squared_log_error
import numpy as np
import random
import csv
import os
 
def setup_seed(seed):
    random.seed(seed)  
    np.random.seed(seed)  
    tf.random.set_seed(seed) 
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    
setup_seed(1)
import sys

import predict_model

In [2]:
# Load data
data = pd.read_excel('IVMD-TN.xlsx',header=None)
S1 = data.values
imfs=S1.swapaxes(1,0)

imfs.shape

(4, 2000)

In [3]:
splita=0.8
splitb=0.9
slength=30
num=10

In [4]:
# Divide the data set. Different method of division from step1.
def DataP(deliver_data,a,b):
    deliver_data = pd.DataFrame(data=deliver_data[0:])
    seq_length = slength
    delay = 1
    data_ = []
    for i in range(len(deliver_data) - seq_length - delay):
        data_.append(deliver_data.iloc[i:i + seq_length + delay])
    data_ = np.array([df.values for df in data_])
    split_a = int(data_.shape[0] * a)
    split_b = int(data_.shape[0] * b)
    
    train_data = data_[:split_a]
    val_data = data_[split_a:split_b]
    test_data=data_[split_b:]
    

    return train_data,test_data,val_data

In [5]:
data1=DataP(imfs[1],splita,splitb)
size=data1[1].shape[0]
size1=data1[0].shape[0]
x={'r2':[111]}

df = pd.DataFrame(data=None)
df.to_csv('imfresult1.csv',mode='w',index=False,header=False)

In [6]:
total_outcome_y = pd.DataFrame(data=0, columns=range(0, 1), index=range(0, size))
total_test_y = pd.DataFrame(data=0, columns=range(0, 1), index=range(0, size))
total_val_yp = pd.DataFrame(data=0, columns=range(0, 1), index=range(0, size1))

In [7]:
list=[]
for i in range(len(imfs)):
    data1=DataP(imfs[i],splita,splitb)

    best=[4,0,2,1]

    if best[i]==0:
        a = predict_model.gru(data1,slength,num)
    elif best[i]==1:
        a = predict_model.lstm(data1,slength,num)
    elif best[i]==2:
        a = predict_model.cnn(data1,slength,num)
    elif best[i]==3:
        a = predict_model.bp(data1,slength,num)
    elif best[i]==4:
        a = predict_model.xgb(data1,slength,num)
    else :
        a = predict_model.svr(data1,slength,num)
    
    
    MAE=mean_absolute_error(a[1], a[0])
    MAPE=mean_absolute_percentage_error(a[1], a[0])
    RMSE=mean_squared_error(a[1], a[0],squared=False) 
    MSE=mean_squared_error(a[1], a[0],squared=True)
    r2=r2_score(a[1], a[0])
    b=a[0].flatten()
    error=abs(a[1]- b)
    STD=np.std(error)
   
    x={'MAE':[MAE],'RMSE':[RMSE],'STD':[STD],'MAPE':[MAPE],'r2':[r2]}
    print(x)
   
    q=a[1].flatten()
    w=a[0].flatten()
    list.append(q)
    list.append(w)
    
    a0 = pd.DataFrame(data=a[0], columns=total_outcome_y.columns, index=total_outcome_y.index)
    a1 = pd.DataFrame(data=a[1], columns=total_test_y.columns, index=total_test_y.index)
    a2 = pd.DataFrame(data=a[2], columns=total_val_yp.columns, index=total_val_yp.index)
    
   
    
    
    total_outcome_y = total_outcome_y + a0
    total_test_y = total_test_y + a1
    total_val_yp=total_val_yp+a2
    

Evaluation metrics for XGB
{'MAE': [0.03216168882093036], 'RMSE': [0.0400192326395836], 'STD': [0.023815221041316347], 'MAPE': [0.00319913785721042], 'r2': [0.9950613817362289]}
50/50 [==============================] - 0s 1ms/step
Evaluation metrics for GRU:
{'MAE': [0.028789106509018177], 'RMSE': [0.04016379130826283], 'STD': [0.02800566868807271], 'MAPE': [0.4708729288264674], 'r2': [0.9818353954248583]}
50/50 [==============================] - 0s 938us/step
Evaluation metrics for CNN:
{'MAE': [0.035244603775318364], 'RMSE': [0.05068555495294332], 'STD': [0.0364258614943923], 'MAPE': [1.038639637785631], 'r2': [0.9337363545144873]}
50/50 [==============================] - 0s 2ms/step
Evaluation metrics for LSTM:
{'MAE': [0.031269241337774104], 'RMSE': [0.04361476127771951], 'STD': [0.03040529472760461], 'MAPE': [1.022285075173886], 'r2': [0.9405116130625502]}


In [8]:
# Evaluate the final forecast results.
data=pd.read_excel('TN.xlsx',header=None)

data2=DataP(data,0.8,0.9)
test_data=data2[1]
test_x = test_data[:, :30, -1]
test_y = test_data[:, -1, -1]
total_outcome_y2=total_outcome_y.values

total_outcome_y3=total_outcome_y2.reshape(len(test_y),)
total_outcome_y4=total_outcome_y3.flatten()
test_y1=test_y.flatten()
error=test_y-total_outcome_y4
error1=abs(error)

MAE=mean_absolute_error(test_y, total_outcome_y)
MAPE=mean_absolute_percentage_error(test_y, total_outcome_y)
RMSE=mean_squared_error(test_y, total_outcome_y,squared=False) 
MSE=mean_squared_error(test_y, total_outcome_y,squared=True)
STD=np.std(error1)

r2=r2_score(test_y, total_outcome_y)
x={'MAE':[MAE],'RMSE':[RMSE],'STD':[STD],'MAPE':[MAPE],'r2':[r2]}

print(x)

{'MAE': [0.11593142521432909], 'RMSE': [0.17314314142833503], 'STD': [0.1286019131717987], 'MAPE': [0.01131137556701886], 'r2': [0.955306633722435]}
